In [123]:
from crewai import Agent, Task, Crew

In [80]:
from textwrap import dedent

In [102]:
from langchain.chat_models import ChatOpenAI

In [114]:
from langchain_ollama.llms import OllamaLLM 

In [81]:
import litellm

In [135]:
import os
from litellm import completion

In [82]:
# Defining our Agents

In [83]:
from langchain_community.llms import Ollama

In [144]:
from litellm import completion

In [175]:
class Travel_Agents:
    def __init__(self):
        self.llm = OllamaLLM(model="deepseek-r1:1.5b")
        self.Ollama = Ollama(model="deepseek-r1:1.5b")
    def expert_travel_agent(self):
        return Agent(
            role = "Expert travel agent",
            backstory = dedent(
                f"""Expert in travel planning and logistics
                i have decades of experience in making travel plan"""
            ),
            goal = dedent(
                f"""ceate a 7-day travel itenarary with detailed per-day plans
                include budget,packing,suggestions and safety tips"""
            ),
            tools=[SearchTools.search_internet, CalculatorTools.calculate],
            allow_delegation = False,
            verbose = True,
            llm = self.llm
        )
        
    def city_selection_expert(self):
        return Agent(
            role = "City Selection Expert",
            backstory = dedent (
                f"""Expert at analyzing travel data to pick ideal destinations"""
            ),
            goal = dedent(
                f"""Select the best cities based on weather, season, prices, and traveler interests"""
            ),
           tools=[SearchTools.search_internet],
           verbose = True,
           llm = self.llm
       )

    def local_tour_guide(self):
        return Agent(
            role = "Local Tour Guide",
            backstory = dedent (
                f"""Knowledgeable local guide with extensive information
                    about the city, it's attractions and customs"""),
            goal = dedent(
                 f"""Provide the BEST insights about the selected city"""),
            tools=[SearchTools.search_internet],
            verbose = True,
            llm = self.llm,
        )

In [85]:
from langchain.tools import tool

In [86]:
# calulatortools are helpful for our agents

In [87]:
class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        """Useful to perform any mathematical calculations
            like sum,minus,multiplication,division,etc
            The input to this tool should be a mathematical expression,a
            couple examples are '200*7','5000/2*10'"""
        try:
            return eval(calculation)
        except SyntaxError:
            return "error:invalid calcualtion expressions"
        

In [88]:
import requests

In [90]:
# search tools helpful for the agent to search on the web

In [91]:
class SearchTools():
    @tool("search the internet")
    def search_internet(query):
        """Useful to search the internet about a given topic and return the results"""
        top_results_to_return = 4
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q":query})
        headers = {
            'X_API_KEY' : serper_api_key,
            'content_type' : "application/json"
        }
        response = requests.request(
            "post",url,headers = headers,data = payload
        )
        if 'oragnic' not in response.json():
            return "Sorry, I couldn't find anything about that, there could an error with you serper api key"
        else:
            results = response.json()['organic']
            string = []
            for result in results[:top_results_to_return]:
                try:
                    string.append('\n'.json([f"title: {result['title']}",f"link: {result['link']}",
                    f"snippet:{result['snippet']}","\n---------"]))
                except KeyError:
                    next
            return '\n'.json(string)
            

In [92]:
class Travel_Plan_Tasks:
    def __tip_selection(self):
        return "If you do your best, I will give you a $10,000 commision"
        
    def plan_itinerary(self,agent,origin,city,interests,travel_dates):
        return Task(
            description = dedent(
                f"""
                **Task**: Develop a 7-Day Travel Itinerary
                **Description**:Expand the city guide into a full 7-day travel itinerary with detailed
                                per-day plans, including weather forecasts, places to eat, packing suggestions,
                                and a budget breakdown. You MUST suggest actual places to visit, actual hotels to stay,
                                and actual restaurants to go to. This itinerary should cover all aspects of the trip,
                                from arrival to departure, integrating the city guide information with practical travel logistics.
                **parameters**:
                - origin : {origin}
                - city : {city}
                - interest : {interests}
                - travel dates : {travel_dates}
                **Note** : {self.__tip_selection}
                """
            ),
            agent = agent,
            expected_output="""
                            A structured 7-day itinerary including:
                            - Day-wise travel plan
                            - Places to visit, accommodations, and dining options
                            - Weather forecasts and packing suggestions
                            - Budget breakdown and estimated costs
                            """

        )
    def identify_city(self, agent, origin, city, interests, travel_dates):
        return Task(
            description = dedent(
                f"""
                **Task**: Identify the Best City for the Trip
                **Description**: Analyze and select the best city for the trip based on specific
                                criteria such as weather patterns, seasonal events, and travel costs.
                                This task involves comparing multiple cities, considering factors like current weather
                                conditions, upcoming cultural or seasonal events, and overall travel expenses.
                                Your final answer must be a detailed report on the chosen city,
                                including actual flight costs, weather forecast, and attractions.
                **parameters**:
                - origin : {origin}
                - city : {city}
                - interests : {interests}
                - travel dates : {travel_dates}
                **Note** : {self.__tip_selection}
                """
            ),
            agent = agent,
            expected_output = dedent(
                """
                A detailed report on the chosen city including:
                - The best city for the trip with justification
                - Current weather forecast
                - Upcoming cultural or seasonal events
                - Estimated flight costs from the origin city
                - Top attractions relevant to the traveler's interests
                - Any additional travel considerations (e.g., visa requirements)
                """
            )
        )

    def gather_info(self, agent, city, interests, travel_dates):
        return Task(
            description = dedent(  # Fixed 'detent' typo
                f"""
                **Task**: Gather In-depth City Guide Information
                **Description**: Compile an in-depth guide for the selected city, gathering information about
                                key attractions, local customs, special events, and daily activity recommendations.
                                This guide should provide a thorough overview of what the city has to offer, including
                                hidden gems, cultural hotspots, must-visit landmarks, weather forecasts, and high-level cost.
                **parameters**:
                - city : {city}
                - interests : {interests}
                - travel dates : {travel_dates}
                """
            ),
            agent = agent,
            expected_output = dedent(
                """
                A comprehensive city guide including:
                - List of key attractions with descriptions
                - Local customs and cultural insights
                - Special events happening during the travel dates
                - Recommended daily activities tailored to the traveler's interests
                - Hidden gems and lesser-known spots
                - Weather forecast for the travel dates
                - High-level cost estimates for accommodations, food, and activities
                """
            )
        )


In [93]:
class TripCrew:
    def __init__(self,origin,city,interests,travel_dates):
        self.origin = origin
        self.city = city
        self.interests = interests
        self.travel_dates = travel_dates
    def run(self):
        # initiating agents and tasks
        agents = Travel_Agents()
        tasks = Travel_Plan_Tasks()

        #defining agents
        expert_travel_agent = agents.expert_travel_agent()
        city_selection_expert = agents.city_selection_expert()
        local_tour_guide = agents.local_tour_guide()

        #definin tasks
        plan_itinerary = tasks.plan_itinerary(expert_travel_agent,self.origin,self.city,self.interests,self.travel_dates)
        identify_city = tasks.identify_city(city_selection_expert,self.origin,self.city,self.interests,self.travel_dates)
        gather_info = tasks.gather_info(local_tour_guide,self.city,self.interests,self.travel_dates)


        #crew
        crew = Crew(
            agents = [expert_travel_agent,city_selection_expert,local_tour_guide],
            tasks = [plan_itinerary,identify_city,gather_info],
            verbose = True
        )

        result = crew.kickoff()
        return results
        

In [122]:
# origin = input("which city are you belonging to?:")
# city = input("Which city do you want to travel to?:")
# interests = input("What are your interests?:")
# travel_dates = input("when do you want to travel?:")
# tripcrew = TripCrew(origin,city,interests,travel_dates)
# result = tripcrew.run()
# print(result)

In [176]:
origin = "chennai"
city = "new zealand"
interests = "dance,sports,swiming"
travel_dates = "25th march 2025"
tripcrew = TripCrew(origin,city,interests,travel_dates)
result = tripcrew.run()
print(result)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to cal


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Expert travel agent
## Task: 
**Task**: Develop a 7-Day Travel Itinerary
**Description**:Expand the city guide into a full 7-day travel itinerary with detailed
                per-day plans, including weather forecasts, places to eat, packing suggestions,
                and a budget breakdown. You MUST suggest actual places to visit, actual hotels to stay,
                and actual restaurants to go to. This itinerary should 

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=[1mOllamaLLM[0m
Params: {}
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers